# **IAA - PRÀCTICA: MAIN**

### **Instal·lar llibreries necessàries**

In [ ]:
%pip install -r ../assets/requirements.txt 

### **Importar llibreries**

In [1]:
def import_dependencies():
	global pd, np, plt, sns, skl

	import pandas as pd
	import numpy as np
	import matplotlib.pyplot as plt
	import seaborn as sns
	import sklearn as skl

import_dependencies()

### **Llegir les dades (Cirrhosis Dataset)**

In [2]:
def load_dataset(save_to_csv: bool = True):
	global data
	from ucimlrepo import fetch_ucirepo 
	
	# Fetch dataset
	cirrhosis_patient_survival_prediction = fetch_ucirepo(id=878)

	data = pd.DataFrame(cirrhosis_patient_survival_prediction.data.original)

	if save_to_csv:
		# Guardem el dataset per poder-lo visualitzar sencer
		data.to_csv('../assets/data/raw_cirrhosis.csv', index=False)

load_dataset(save_to_csv=True)

### **Informació del dataset inicial**

In [ ]:
data.shape

In [ ]:
data.head(-10)

In [ ]:
data.info()

### **Preprocessing inicial**

In [3]:
def initial_preprocessing(data: pd.DataFrame, save_to_csv: bool = True):
	"""
	Reemplaça els valors 'NaNN' per NaN, assigna els tipus de dades correctes a cada columna i renombra les classes d'algunes variables per una millor comprensió.
	"""
	# Reemplaçar l'string 'NaNN' per NaN
	data.replace(to_replace=['NaNN', '', pd.NA], value=np.nan, inplace=True)

	# Assignem els tipus de dades correctes a cada columna
	int64_variables = ['N_Days', 'Age', 'Cholesterol', 'Copper', 'Tryglicerides', 'Platelets']
	float64_variables = ['Bilirubin', 'Albumin', 'Alk_Phos', 'SGOT', 'Prothrombin']
	category_variables = ['ID', 'Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage']
	boolean_variables = ['Ascites', 'Hepatomegaly', 'Spiders']

	data[int64_variables] = data[int64_variables].astype('Int64')
	data[float64_variables] = data[float64_variables].astype('float64')
	data[category_variables] = data[category_variables].astype('category')


	# Renombrem les classes d'algunes variables per una millor comprensió
	data['Status'] = data['Status'].replace({'D': 'Dead', 'C': 'Alive', 'CL': 'LiverTransplant'})
	data[boolean_variables] = data[boolean_variables].replace({'Y': 1, 'N': 0})
	data['Edema'] = data['Edema'].replace({'N': 'NoEdema', 'S': 'EdemaResolved', 'Y': 'EdemaPersistent'})

	if save_to_csv:
		# Guardem el dataset
		data.to_csv('../assets/data/initial_preprocessing_cirrhosis.csv', index=False)

initial_preprocessing(data=data, save_to_csv=True)

In [ ]:
data.head(-10)

In [ ]:
data.info()

### **Anàlisis inicial de les variables**

In [ ]:
data.head(-10)

In [ ]:
data.isna().sum().sort_values(ascending=False)

In [ ]:
# Estudi de les variables numèriques
data.describe()

In [ ]:
# Estadístiques de les variables categòriques
data.describe(include='category')

In [ ]:
def numerical_vars_histograms(data: pd.DataFrame):
    # Visualització de les distribucions de les variables numèriques en una sola figura
    numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

    num_rows = int(np.ceil(len(numerical_columns) / 2))

    fig = plt.figure(figsize=(10, num_rows * 4))

    for i, col in enumerate(numerical_columns):
        ax = fig.add_subplot(num_rows, 2, i + 1)
        
        sns.histplot(data[col], edgecolor="k", linewidth=1.5, kde=True)
        
        plt.xticks(rotation=45, ha='right')
        
        ax.set_title(f'Distribució de la variable numèrica {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Freqüència')

    plt.tight_layout()
    plt.show()

numerical_vars_histograms(data=data)

In [ ]:
def categorical_vars_countplots(data: pd.DataFrame):
    """
    Visualització de les distribucions de les variables categòriques en una sola figura (menys ID).
    """
    # Visualització de les distribucions de les variables categòriques en una sola figura (menys ID)
    categorical_columns = data.select_dtypes(include=['category']).columns.drop(['ID'])
    num_rows = int(np.ceil(len(categorical_columns) / 2))

    fig = plt.figure(figsize=(10, num_rows * 4))

    for i, col in enumerate(categorical_columns):
        ax = fig.add_subplot(num_rows, 2, i + 1)
        
        sns.countplot(data=data, x=col, ax=ax, hue=col, legend=False)
        
        plt.xticks(rotation=45, ha='right')
        
        ax.set_title(f'Distribució de la variable categòrica {col}')
        ax.set_xlabel(col)
        ax.set_ylabel('Quantitat')

    plt.tight_layout()
    plt.show()

categorical_vars_countplots(data=data)

### **Tractament d'outliers**

In [ ]:
def compare_iqr_factors(data: pd.DataFrame, factors: list = [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]):
	"""
	Compara diferents factors que multipliquen al IQR per a determinar els outliers i realitza un gràfic evolutiu per comparar-los.
	"""
	numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

	plt.figure(figsize=(10, 6))

	# Dictionary to store outlier percentages for each factor and column
	outlier_percentages = {col: [] for col in numerical_columns}
	total_percentages = [set() for _ in range(len(factors))]

	for col in numerical_columns:
		Q1 = data[col].quantile(0.25)
		Q3 = data[col].quantile(0.75)
		IQR = Q3 - Q1

		for f_id, factor in enumerate(factors):
			outliers_mask = ((data[col] < (Q1 - factor * IQR)) | (data[col] > (Q3 + factor * IQR)))
			total_percentages[f_id].update(data.index[outliers_mask])
			outliers_percentage = np.mean(outliers_mask) * 100
			outlier_percentages[col].append(outliers_percentage)

	total_percentages = [(len(outliers) / len(data)) * 100 for outliers in total_percentages]
			
	# Plotting the results
	for col, percentages in outlier_percentages.items():
		plt.plot(factors, percentages, label=col)
	plt.plot(factors, total_percentages, label='Total', linestyle='--', color='black')

	plt.xlabel('Factor multiplicatiu del IQR')
	plt.ylabel('Percentage d\'outliers (%)')
	plt.title('Percentatge d\'outliers de cada variable numèrica per a diferents factors multiplicatius del IQR')
	plt.legend()
	plt.grid(True)
	plt.show()

compare_iqr_factors(data=data, factors=[1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

In [4]:
def delete_outliers(data: pd.DataFrame, factor: float = 1.5, plots: bool = True, save_to_csv: bool = True):
    """
    Funció que detecta, visualitza i elimina els outliers d'un dataset. El factor multiplica el IQR per a determinar quins valors són outliers.
    """
    # Detecció, visualització i eliminació d'outliers
    numerical_columns = data.select_dtypes(include=['Int64', 'float64']).columns

    outliers_indices = []

    for col in numerical_columns:
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers_mask = ((data[col] < (Q1 - factor * IQR)) | (data[col] > (Q3 + factor * IQR)))
        outliers = data[col][outliers_mask]
        non_outliers = data[col][~outliers_mask]

        outliers_indices.extend(data[col][outliers_mask].index.tolist())
        
        if plots:
            fig, axes = plt.subplots(1, 2, figsize=(8, 5))

            # Boxplot amb els outliers originals
            sns.boxplot(ax=axes[0], y=data[col], orient='v')
            axes[0].scatter(x=[0]*len(outliers), y=outliers, color='red', marker='o')
            axes[0].set_title(f'Boxplot de {col} amb outliers ({factor}x IQR)')
            
            percent_outliers = len(outliers) / data.shape[0] * 100
            axes[0].text(0.5, -0.1, f'Outliers ({factor}x IQR): {len(outliers)} ({percent_outliers:.2f}%)', 
                        ha='center', va='center', transform=axes[0].transAxes)
            
            # Boxplot sense els outliers
            sns.boxplot(ax=axes[1], y=non_outliers, orient='v')
            axes[1].set_title(f'Boxplot de {col} sense outliers')

            plt.tight_layout()
            plt.show()

    unique_outliers = len(set(outliers_indices))

    print(f"Datset amb outliers: {data.shape[0]} files i {data.shape[1]} columnes.")
    print(f"Nombre total d'outliers únics eliminats: {unique_outliers} ({unique_outliers / data.shape[0] * 100:.2f}% de tot el dataset).")

    # Eliminació d'outliers
    data.drop(list(set(outliers_indices)), inplace=True)
    
    print(f"Dataset sense outliers: {data.shape[0]} files i {data.shape[1]} columnes.")

    if save_to_csv:
        # Guardem el dataset
        data.to_csv('../assets/data/no_outliers_cirrhosis.csv', index=False)

delete_outliers(data=data, factor=3, plots=False, save_to_csv=True)

Datset amb outliers: 418 files i 20 columnes.
Nombre total d'outliers únics eliminats: 70 (16.75% de tot el dataset).
Dataset sense outliers: 348 files i 20 columnes.


### **Recodificació de variables categòriques**

In [5]:
def encode_variables(data: pd.DataFrame, save_to_csv: bool = True):
    """
    Codifica les variables categòriques que calgui per a poder-les utilitzar en els models de ML. 
    A més, guarda el mapping per a poder decodificar-les.
    Els NaNs es mantenen (en comptes de considerar-los una classe més) per poder imputar-los posteriorment.
    """
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.impute import SimpleImputer

    global ohe_mapping

    columns_to_encode = ['Drug', 'Sex', 'Edema', 'Stage'] # Sense la variable 'Status' perquè és la target i, a més, no té valors NaN

    na_indexs_per_old_encoded_column = {col: set(data[data[col].isna()].index) for col in columns_to_encode} # Guardem els indexs dels NaNs per a cada columna a codificar
    new_encoded_columns_per_old_encoded_column = {col: set() for col in columns_to_encode} # Guardem les classes de cada columna a codificar

    # Imputem els NaNs per evitar que es crein columnes innecessàries al fer el OneHotEncoding. Després tornarem a inserir els NaNs
    data[columns_to_encode] = SimpleImputer(strategy='most_frequent').fit_transform(data[columns_to_encode])

    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

    data_encoded = ohe.fit_transform(data[columns_to_encode])
    encoded_columns = ohe.get_feature_names_out(columns_to_encode)

    # Guardem el mapping per a poder decodificar les variables
    ohe_mapping = {}
    for i, col in enumerate(columns_to_encode):
        for category in ohe.categories_[i]:
            new_encoded_column_name = f"{col}_{category}"
            ohe_mapping[new_encoded_column_name] = (col, category)
            new_encoded_columns_per_old_encoded_column[col].add(new_encoded_column_name)

    data[encoded_columns] = data_encoded
    data.drop(columns_to_encode, axis=1, inplace=True)

    # Tornem a posar els NaNs per poder imputar-los
    for col in columns_to_encode:
        for na_index in na_indexs_per_old_encoded_column[col]:
            for new_column in new_encoded_columns_per_old_encoded_column[col]:
                data.loc[na_index, new_column] = np.nan

    if save_to_csv:
        # Guardem el dataset
        data.to_csv('../assets/data/encoded_cirrhosis.csv', index=False)

encode_variables(data=data, save_to_csv=True)

### **Partició del dataset en train/test**

In [ ]:
def split_dataset(data: pd.DataFrame, test_size: float = 0.15, random_state: int = 42):
	"""
	Particiona el dataset en train i test.
	"""
	global X_train, y_train, test

	from sklearn.model_selection import train_test_split

	train, test = train_test_split(data, test_size=test_size, random_state=random_state)

	print(f"Train shape: {train.shape}")
	print(f"Test shape: {test.shape}")

	# 'Status' és la variable target
	X_train = train.drop(columns=['Status'])
	y_train = train['Status']

split_dataset(data=data, test_size=0.15, random_state=42)

### **Imputar els valors faltants (Missings)**

In [ ]:
data.isna().sum().sort_values(ascending=False)

In [7]:
def decode_variables(data: pd.DataFrame, ohe_mapping):
    # Invertir el mapping per facilitar la decodificació
    inv_ohe_mapping = {}
    for new_col, (orig_col, _) in ohe_mapping.items():
        inv_ohe_mapping.setdefault(orig_col, []).append(new_col)

    print(inv_ohe_mapping)

    # Creem les columnes decodificades
    for orig_col, new_cols in inv_ohe_mapping.items():
        # Extraure la columna amb el valor 1 (hot)
        data[orig_col] = data[new_cols].idxmax(axis=1)
        .apply(lambda new_col_name: new_col_name.split('_')[-1] if new_col_name != np.nan else np.nan)
        # CONTINUAR AQUI. CAL PASSAR ELS VALORS DE COLUMNES ANTIGUES A NOUS
        # Eliminar les columnes codificades
        data.drop(new_cols, axis=1, inplace=True)

decode_variables(data=data, ohe_mapping=ohe_mapping)

{'Drug': ['Drug_D-penicillamine', 'Drug_Placebo'], 'Sex': ['Sex_F', 'Sex_M'], 'Edema': ['Edema_EdemaPersistent', 'Edema_EdemaResolved', 'Edema_NoEdema'], 'Stage': ['Stage_1.0', 'Stage_2.0', 'Stage_3.0', 'Stage_4.0']}


C:\Users\Cai Selvas Sala\AppData\Local\Temp\ipykernel_14520\1957590651.py:12: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  data[orig_col] = data[new_cols].idxmax(axis=1).apply(lambda new_col_name: new_col_name.split('_')[-1] if new_col_name != np.nan else np.nan)


AttributeError: 'float' object has no attribute 'split'

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.impute import KNNImputer, IterativeImputer, SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_squared_error
import numpy as np

def mejor_imputacion(X_train, y_train):
    na_columns = X_train.isna().any()

    numerical_columns = X_train.select_dtypes(include=['Int64', 'float64']).columns
    categorical_columns = X_train.select_dtypes(include=['category']).columns.drop(['ID'])

    MixedImputer = ColumnTransformer([
        ('imputer', SimpleImputer(strategy='mean'), numerical_columns),
        ('imputer', SimpleImputer(strategy='most_frequent'), categorical_columns)
    ])

    imputaciones = {
        'mixed': MixedImputer, # Mean per numèriques, most_frequent per categòriques
        'knn-3': KNNImputer(n_neighbors=3),
        'knn-5': KNNImputer(n_neighbors=5),
        'iterative-10': IterativeImputer(max_iter=10, random_state=42),
        'iterative-20': IterativeImputer(max_iter=20, random_state=42),
    }

    scores = {}
    best_score = float('inf')
    best_imputer = None

    # Generar valores NA artificialmente
    X_missing = X_train.copy()
    for col in X_missing.columns:
        X_missing.loc[X_missing.sample(frac=0.1).index, col] = np.nan

    for name, imputer in imputaciones.items():
        X_imputed = imputer.fit_transform(X_missing)
        score = mean_squared_error(X_train, X_imputed)

        scores[name] = score
        if score < best_score:
            best_score = score
            best_imputer = name

    return best_imputer, scores

mejor_imputador, puntuaciones = mejor_imputacion(X_train, y_train)


def cross_validation_con_imputacion(X, y):
    kf = KFold(n_splits=5)  # Ajustar según necesidades

    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        imputacion = mejor_imputacion(X_train, y_train)
        # Aplicar la mejor imputación y entrenar el modelo...
        # Evaluar el modelo en X_test, y_test...


In [ ]:
def impute_missings(data: pd.DataFrame, save_to_csv: bool = True):
    from sklearn.impute import KNNImputer
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.compose import ColumnTransformer

    categorical_columns = data.select_dtypes(include=['category']).columns.drop(['ID'])

    # Creación del transformador para la codificación One-Hot
    one_hot_encoder = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)
        ],
        remainder='passthrough'
    )

    # Aplicación de la codificación One-Hot
    data_encoded = one_hot_encoder.fit_transform(data)

    # Creación del imputador KNN
    knn_imputer = KNNImputer(n_neighbors=5)

    # Imputación de los datos codificados
    data_imputed = knn_imputer.fit_transform(data_encoded)

    # Convirtiendo de nuevo a un DataFrame de pandas (opcional, dependiendo de cómo necesites usar los datos)
    # Nota: La conversión de nuevo a DataFrame requiere asignar nombres a las columnas
    # Esto puede ser complejo debido a la codificación One-Hot
    data_imputed_df = pd.DataFrame(data_imputed)

    # Mostrando las primeras filas de los datos imputados
    data_imputed_df.head()

    if save_to_csv:
        # Guardem el dataset
        data_imputed_df.to_csv('../assets/data/imputed_cirrhosis.csv', index=False)


In [ ]:
from sklearn.impute import SimpleImputer
num_columns = X_train.select_dtypes(include=['float64', 'int64']).columns
cat_columns = X_train.select_dtypes(include=['object']).columns
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')
X_train[num_columns] = imputer_num.fit_transform(X_train[num_columns])
X_train[cat_columns] = imputer_cat.fit_transform(X_train[cat_columns])
X_test[num_columns] = imputer_num.transform(X_test[num_columns])
X_test[cat_columns] = imputer_cat.transform(X_test[cat_columns])

# Codificación one-hot de variables categóricas
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = pd.DataFrame(encoder.fit_transform(X_train[cat_columns]))
X_train_encoded.columns = encoder.get_feature_names_out(cat_columns)
X_train = X_train.drop(cat_columns, axis=1)
X_train = pd.concat([X_train, X_train_encoded], axis=1)
X_test_encoded = pd.DataFrame(encoder.transform(X_test[cat_columns]))
X_test_encoded.columns = encoder.get_feature_names_out(cat_columns)
X_test = X_test.drop(cat_columns, axis=1)
X_test = pd.concat([X_test, X_test_encoded], axis=1)

**1r Model: K-Nearest Neighbors (KNN)**

**2n Model: Decision Tree**

**3r Model: Support Vector Machine (SVM)**